In [1]:
import sys

modules_dir = 'D:/usr14/project/Binary/adiabaticbinary/ww'
sys.path.insert(0, modules_dir)

In [2]:
from modules import resnet1
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import numpy as np
from readdata import read_dataset

In [3]:
device=torch.device('cuda')
def fit(model:resnet1.ResNet, optim,lossfunc,trainloader: DataLoader):
    model.train()
    totalloss = 0
    for data, target in trainloader:
        data, target = data.to(device), target.to(device)
        optim.zero_grad()
        output = model(data).to(device)
        loss = lossfunc(output, target)
        loss.backward()
        optim.step()
        with torch.no_grad():
            totalloss += loss.item()*data.size(0)
    return totalloss/len(trainloader.sampler)

def evaluate(model:resnet1.ResNet, val_loader: DataLoader,lossfunc:nn.CrossEntropyLoss):
    model.eval()
    loss=0
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs).to(device)
            loss+=lossfunc(outputs,labels).item()*inputs.size(0)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct+=(predicted==labels).sum().item()

    return loss/len(val_loader.sampler) ,correct / total

In [4]:
model = resnet1.ResNet(False, False).to(device)
lossfunc=nn.CrossEntropyLoss().to(device)
train_loader,val_loader,test_loader=read_dataset(64)

In [5]:
lr=0.1
counter=0
min_val_loss=np.inf
for epoch in range(150):
    if counter/10==1:
        counter=0
        lr*=0.5
        print(f"lr reduced to {lr}")
    optim=torch.optim.SGD(model.parameters(), lr=lr, momentum=0.9, weight_decay=5e-4)
    loss=fit(model,optim,lossfunc,train_loader)
    val_loss,val_acc=evaluate(model,val_loader,lossfunc)
    print(f'epoch: {epoch+1}, loss: {loss}, val_loss: {val_loss}, val_acc:{val_acc}')
    if val_loss<min_val_loss:
        min_val_loss=val_loss
        print(f"val loss reduced to {min_val_loss}")
        counter=0
        torch.save(model.state_dict(), 'pre1.pth')
    else:
        counter+=1

epoch: 1, loss: 2.212570424079895, val_loss: 1.8024646927760197, val_acc:0.2696153846153846
val loss reduced to 1.8024646927760197
epoch: 2, loss: 1.7664664088762723, val_loss: 1.72707272199484, val_acc:0.3403846153846154
val loss reduced to 1.72707272199484
epoch: 3, loss: 1.6745480023897612, val_loss: 1.637961918757512, val_acc:0.39076923076923076
val loss reduced to 1.637961918757512
epoch: 4, loss: 1.616458020088, val_loss: 1.6346203605945293, val_acc:0.3776923076923077
val loss reduced to 1.6346203605945293
epoch: 5, loss: 1.5772594547271728, val_loss: 1.5165754674031184, val_acc:0.4123076923076923
val loss reduced to 1.5165754674031184
epoch: 6, loss: 1.5275495049892327, val_loss: 1.5219117359014658, val_acc:0.4269230769230769
epoch: 7, loss: 1.4796075236491668, val_loss: 1.4733678216200607, val_acc:0.4492307692307692
val loss reduced to 1.4733678216200607
epoch: 8, loss: 1.460910730973268, val_loss: 1.563189444908729, val_acc:0.40615384615384614
epoch: 9, loss: 1.427959325741498